In [1]:
from dotenv import load_dotenv

load_dotenv()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [2]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Chinese"),
    HumanMessage(content="hi!"),
]

llm.invoke(messages)

AIMessage(content='嗨！', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 20, 'total_tokens': 24}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-beae1f49-2875-4d34-aaa6-f2ebd0b924ca-0', usage_metadata={'input_tokens': 20, 'output_tokens': 4, 'total_tokens': 24})

In [3]:
import chromadb
from chromadb import Settings

from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

from config import CHROMA_DATA_PATH,CHROMA_TENANT, CHROMA_DATABASE

CHROMA_CLIENT = chromadb.PersistentClient(
        path=CHROMA_DATA_PATH,
        settings=Settings(allow_reset=True, anonymized_telemetry=False),
        tenant=CHROMA_TENANT,
        database=CHROMA_DATABASE,
    )

collection_name = "c1b6bfaae0e0a5cadf2eb1fc0928739ee7bb60fd96170a7f839a2fcb0738ecd"
# Create LangChain Chroma instance
langchain_chroma = Chroma(
    client=CHROMA_CLIENT,
    collection_name=collection_name,
    embedding_function=OpenAIEmbeddings(),  # Replace with your preferred embedding function
)

INFO:config:Embedding model set: text-embedding-3-small


d:\appChat_demo


In [4]:
documents = langchain_chroma.get()

# all_documents = langchain_chroma.get_all()

print(documents)

{'ids': ['d9809c1e-c1ec-48c5-a17f-2500fe5c1272'], 'embeddings': None, 'metadatas': [{'source': 'D:\\appChat_demo\\backend\\data\\docs\\tesla.txt', 'start_index': 0}], 'documents': ['Tesla Motors is an American electric sports car manufacturer based in Silicon Valley, California and founded by former CEO Martin Eberhard. Their first production vehicle, the Tesla Roadster, is a performance sports car. According to the company, the car has a range of 200 miles, acceleration from 0 to 60 mph in 4 seconds, and a top speed of 130 mph. Prototypes were introduced to the public in July 2006, and the first production models are planned for summer 2007. Demand was high for the first \'Signature One Hundred\' Roadsters, which sold out in less than three weeks. According to a note on the company\'s web site, as of November 2006 the full run of the "2007 1/2" model have been sold out, constituting 220+ cars. The company also planned to offer home roof mounted solar-photovoltaic systems that will off

In [5]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

retriever = langchain_chroma.as_retriever()

prompt = hub.pull("rlm/rag-prompt")

query = "summarize the content in 20 words"


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
context = rag_chain.invoke(query)

In [6]:
print(context)

Tesla Motors is an American electric sports car manufacturer known for its Tesla Roadster, which offers high performance and range. The company also planned to offer solar-photovoltaic systems for home charging, making it "energy positive." The Roadster prototypes were introduced in July 2006, with the first production models planned for summer 2007.
